In [1]:
import polars as pl
import spacy
import os
from sklearn.feature_extraction.text import TfidfVectorizer

pl.Config(tbl_width_chars=1000)

In [4]:
DATASET = f"{os.getcwd()}/data/base_dataset.parquet"
DATASET

'/Users/diegolopes/repositories/geo-nlq-to-sql/data/base_dataset.parquet'

In [5]:
df = pl.scan_parquet(DATASET)
df

In [137]:
df.head().collect()

id,question,territorial_division,level,geospatial_functions,sql_code
i64,str,str,str,str,str
1,"""Quais escolas urbanas ficam em…","""País""","""Facíl""","""ST_Intersects""","""SELECT e.cd_entidade, m.cd…"
2,"""Quais bibliotecas com CEP ause…","""País""","""Facíl""","""ST_Intersects""","""SELECT b.cd_bib, b.nm_bib,…"
3,"""Quais CRAS dos estados SP e PR…","""País""","""Facíl""","""ST_Intersects""","""SELECT c.nu_identif, c.nm_…"
4,"""Quais CREAS com CEP preenchido…","""País""","""Facíl""","""ST_Intersects""","""SELECT r.nu_identif, r.nm_…"
5,"""Relacione grandes regiões e se…","""País""","""Facíl""","""ST_Intersects""","""SELECT r.cd_regia, r.nm_re…"


In [129]:
df.write_parquet(f"{os.getcwd()}/data/dataset_base.parquet")

In [63]:
nlp = spacy.load('pt_core_news_lg')

In [65]:
documents = []

for row in df.iter_rows():
    documents.append(nlp(row[1]))

In [38]:
corpus = df.collect()['question'].to_list()
vectorizer = TfidfVectorizer(stop_words=['?'])
X = vectorizer.fit_transform(corpus)

In [13]:
type(X)

scipy.sparse._csr.csr_matrix

In [116]:
vectorizer.vocabulary_['urbanas']

1158

In [39]:
feature_names = vectorizer.get_feature_names_out()
type(feature_names)

numpy.ndarray

In [117]:
X[0, 1158]

np.float64(0.36136261353612775)

In [40]:
doc = 0
feature_index = X[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [X[doc, x] for x in feature_index])

In [21]:
tfidf_scores

In [17]:
pl.DataFrame(tfidf_scores)

column_0,column_1
i64,f64
930,0.156042
444,0.105574
1158,0.361363
492,0.210794
424,0.137929
…,…
622,0.364332
758,0.295955
574,0.479705


In [43]:
mapping = [(w,s) for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]]

mapping

[]

In [9]:
l1 = [1,2,3,4]
l2 = [5,6,7,8]

list(zip(l1,l2))

[(1, 5), (2, 6), (3, 7), (4, 8)]